In [0]:
spark.conf.set(
    "fs.azure.account.key.customerdatamlfile.blob.core.windows.net","YBz6dOitJZ04GWH+eKOep1hTspIE5q4o2anTkit8BRHOFbrcVT2pzswe37PUWw6VoYlD5OWef4JK+AStDU5D1Q==")

In [0]:
dbutils.fs.ls("wasbs://rawdata@customerdatamlfile.blob.core.windows.net/")


[FileInfo(path='wasbs://rawdata@customerdatamlfile.blob.core.windows.net/car_data.csv', name='car_data.csv', size=22176, modificationTime=1735697145000)]

In [0]:
spark_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("wasbs://rawdata@customerdatamlfile.blob.core.windows.net/car_data.csv")
spark_df.show()


+-------+------+---+------------+---------+
|User ID|Gender|Age|AnnualSalary|Purchased|
+-------+------+---+------------+---------+
|    385|  Male| 35|       20000|        0|
|    681|  Male| 40|       43500|        0|
|    353|  Male| 49|       74000|        0|
|    895|  Male| 40|      107500|        1|
|    661|  Male| 25|       79000|        0|
|    846|Female| 47|       33500|        1|
|    219|Female| 46|      132500|        1|
|    588|  Male| 42|       64000|        0|
|     85|Female| 30|       84500|        0|
|    465|  Male| 41|       52000|        0|
|    686|  Male| 42|       80000|        0|
|    408|  Male| 47|       23000|        1|
|    790|Female| 32|       72500|        0|
|    116|Female| 27|       57000|        0|
|    118|Female| 42|      108000|        1|
|     54|Female| 33|      149000|        1|
|     90|  Male| 35|       75000|        0|
|    372|  Male| 35|       53000|        0|
|    926|  Male| 46|       79000|        1|
|     94|Female| 39|      134000

In [0]:
df = spark_df.toPandas()

In [0]:
df.head()

,User ID,Gender,Age,AnnualSalary,Purchased
0,385,Male,35,20000,0
1,681,Male,40,43500,0
2,353,Male,49,74000,0
3,895,Male,40,107500,1
4,661,Male,25,79000,0


In [0]:
df.dtypes

User ID          int32
Gender          object
Age              int32
AnnualSalary     int32
Purchased        int32
dtype: object

In [0]:
df.isnull().sum()

User ID         0
Gender          0
Age             0
AnnualSalary    0
Purchased       0
dtype: int64

In [0]:
# car purchase 0 = no and 1 = yes
# drop user id
df = df.drop('User ID',axis = 1)

In [0]:
df.head()

,Gender,Age,AnnualSalary,Purchased
0,Male,35,20000,0
1,Male,40,43500,0
2,Male,49,74000,0
3,Male,40,107500,1
4,Male,25,79000,0


In [0]:
# encode male as 1 and female as 0
df['Gender'] = df["Gender"].apply(lambda x:1 if x=='Male' else 0)

In [0]:
df.head()

,Gender,Age,AnnualSalary,Purchased
0,1,35,20000,0
1,1,40,43500,0
2,1,49,74000,0
3,1,40,107500,1
4,1,25,79000,0


In [0]:
# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)


In [0]:
spark_df.show(5)

+------+---+------------+---------+
|Gender|Age|AnnualSalary|Purchased|
+------+---+------------+---------+
|     1| 35|       20000|        0|
|     1| 40|       43500|        0|
|     1| 49|       74000|        0|
|     1| 40|      107500|        1|
|     1| 25|       79000|        0|
+------+---+------------+---------+
only showing top 5 rows



In [0]:
print(f"Number of rows in the DataFrame: {spark_df.count()}")


Number of rows in the DataFrame: 1000


In [0]:
# Repartition to 1 to ensure a single partition
spark_df.repartition(1) \
         .write \
         .option("header", "true") \
         .csv("wasbs://cleandata@customerdatamlfile.blob.core.windows.net/clean_car_data/")
